In [1]:
def correct_input(n,state):
    # Checks array has n rows
    if len(state)!=n:
        return False
    
    # Checks array has n columns
    for arr in state:
        if len(arr)!=n:
            return False

    # Flattens list for easier comparisons
    flat_list = [number for sublist in state for number in sublist]

    # Sorts list to check it's the same as a list from 0 to n^2.
    if sorted(flat_list) != list(range(n**2)):
        return False
    return True


In [2]:
import numpy as np
import math
import heapq

# A* Search Example for Robot Navigation
# R. Shekhar
# August 10, 2017



# Define the heuristic functions here
def misplaced_heur(state):
    """
    Returns the amount of tiles that aren't in their correct position.
    """
    val = 0
    # This line flattens the state into a single list.
    flat_list = [number for sublist in state for number in sublist]
    for i in range(1,len(flat_list)):
        # The tile for i should be in i-eth position of the flat list.
        if flat_list[i] != i:
            val +=1
    return val

def manhattan_heur(state):
    """
    Returns the sum of the manhattan distance between the current position of
    a tile and their goal position.
    """
    val = 0
    # Creates the goal state to compare to it.
    goal = (np.array(list(range(len(state)**2))).reshape(len(state),len(state)))
    for i in range(len(state)):
        for j in range(len(state)):
            # Identifies the number in position [i][j]
            number = state[i][j]
            # 0 is not a tile
            if number == 0:
                continue
            # Looks up where the number should be.
            a,b = np.where(goal==number)
            # Adds the manhattan distance
            val += math.fabs(a-i)+math.fabs(b-j)
    return val

heuristics = [misplaced_heur,manhattan_heur]
    
#Current heuristic function. Change the handle cur_heur to point to the heuristic function you want to test
cur_heur = heuristics[1];


def get_next_states(state):
    """
    Returns list of lists with the states you can move to from
    the current state.
    """
    # get row and column of the empty piece
    y, x = np.where(state==0)
    moves = []
    # find which pieces can move there
    if y > 0:
        copy = np.copy(state)
        copy[y,x]=state[y-1,x]
        copy[y-1,x]=0
        moves.append(copy)
    if x > 0:
        copy = np.copy(state)
        copy[y,x]=state[y,x-1]
        copy[y,x-1]=0
        moves.append(copy)
    if y < 2:
        copy = np.copy(state)
        copy[y,x]=state[y+1,x]
        copy[y+1,x]=0
        moves.append(copy)
    if x < 2:
        copy = np.copy(state)
        copy[y,x]=state[y,x+1]
        copy[y,x+1]=0
        moves.append(copy) 
    return moves

def gen_solution_path(node, path=[]):
    """Generates a list where each Node has its parent afterwards."""
    path.append(node)
    if node.parent == None:
        return path
    else:
        return gen_solution_path(node.parent,path)

    
def print_path(path):
    """
    Takes a list that should contain nodes and prints their states 
    enumerating the steps.
    """
    step = 0
    for node in path:
        print("Step ", step, ":")
        step += 1
        node.__str__()
        print("\n")

def flatten(state):
    """Flattens a list of lists into a single list."""
    return [number for sublist in state for number in sublist]

# Define the data structure for A* search node
class PuzzleNode:
    """
    Defines the puzzle class.
  
    Class Instance Atrributes:
        parent(PuzzleNode object): Refers to the parent of the node.
        state(numpy array): Stores the state of the node.
        fval(int): Stores the sum of the gval and the heuristic value of the state.
        gval(int): Counts the amount of steps required to get from the 
                   original node to this node.
        pruned(bool): If true, this node has already been explored
    """ 
    def __init__(self,state,fval,gval,parent=None):
        self.state = np.array(state)
        self.fval = fval
        self.gval = gval
        self.parent = parent
        self.pruned = False

    # Comparison function based on f cost
    def __lt__(self,other):
        return self.fval < other.fval

    # Convert to string
    def __str__(self):
        for row in self.state:
            print(row)

def solvePuzzle(n,start,heuristic=cur_heur,prnt=False, _counter = False):
    """
    The function implements an A* search to reach the solution of a puzzle.
    The first half of the function checks it has an appropriate shape and that
    the tiles do not have repeated numbers.
    
    Parameters:
        n(int): Tells us how many rows and columns we should expect in the state.
        state(list): A list containing every row of the initial state of 
                     the puzzle from top to bottom.
        heuristic(func): Defines how we evaluate the state for the priority we give it.
        prnt(bool): If True, prints out the maximum frontier size, the amount of steps
                    it took to get to the goal state, and every step from the initial
                    state to the goal state.
        _counter(bool): If True, returns a fourth argument with the number of nodes
                        expanded before reaching the solution.
    """
    # Initializes the values that will be returned.
    steps = 0
    frontierSize = 0
    err = 0
    
    # Checks the input is correct to return an error if it's not.
    if not correct_input(n,start):
        err = -1
        return steps, frontierSize, err
    
    # Goal state
    goal = np.array(list(range(n**2))).reshape(n,n)
    
    # Keeps track of the maximum size of the frontier so far.
    frontierSize = 0
    
    # Start node
    start_node = PuzzleNode(start,heuristic(start),0)
    
    # Dictionary with current cost to reach all visited nodes
    costs_db = {tuple(flatten(start)):start_node}
    
    # This change into a queue was to be able to use the len() function
    # and measure frontier size
    frontier = []
    heapq.heappush(frontier,(start_node,0))
    
    # Begin A* Search
    expansion_counter = 0
    
    while frontier:
        # Update to check the maximum size of the frontier
        frontierSize = max(frontierSize,len(frontier))
        
        # Take the next available node from the priority queue
        cur_node, counter = heapq.heappop(frontier)
    
        if cur_node.pruned:
            continue # Skip if this node has been marked for removal
    
        # Check if we are at the goal
        if (cur_node.state == goal).all(): 
            break
        
        # Gets next possible states
        next_moves = get_next_states(cur_node.state)

        # Increase the count of expansions
        expansion_counter = expansion_counter + 1

        # Expand the node
        for next_state in next_moves:
            #This is turned into a tuple so that it will be hashable to find in dict.
            flat_state = tuple(flatten(next_state))
            gval = cur_node.gval + 1 # Tentative cost value for child
            
            # If the child node is already in the cost database (i.e. explored)
            if flat_state in costs_db:
                
                if costs_db[flat_state].gval > gval:
                    costs_db[flat_state].pruned = True # Mark existing value for deletion from frontier
                else:
                    continue # ignore this child, since a better path has already been found previously.
    
            hval = heuristic(next_state) # Heuristic cost from next node to goal
            next_node = PuzzleNode(next_state,gval+hval,gval,cur_node) # Create new node for child
            heapq.heappush(frontier,(next_node,expansion_counter))
            costs_db[flat_state] = next_node #Mark the node as explored

    path = gen_solution_path(cur_node,path=[])
    path.reverse()
    steps = len(path)-1
    if prnt == True:
        print("A* search completed in %d steps\n"% expansion_counter)
        print("Optimal Path to Goal:\n")
        print_path(path)
        print("Steps to solution: ", steps)
        print("Maximum frontier size: ",frontierSize)
    
    if _counter == True:
        return steps, frontierSize, err, expansion_counter
    return steps, frontierSize, err

In [3]:
solvePuzzle(3,[[2,3,7],[1,8,0],[6,5,4]],heuristic=heuristics[1],prnt=True)

A* search completed in 83 steps

Optimal Path to Goal:

Step  0 :
[2 3 7]
[1 8 0]
[6 5 4]


Step  1 :
[2 3 7]
[1 8 4]
[6 5 0]


Step  2 :
[2 3 7]
[1 8 4]
[6 0 5]


Step  3 :
[2 3 7]
[1 0 4]
[6 8 5]


Step  4 :
[2 0 7]
[1 3 4]
[6 8 5]


Step  5 :
[0 2 7]
[1 3 4]
[6 8 5]


Step  6 :
[1 2 7]
[0 3 4]
[6 8 5]


Step  7 :
[1 2 7]
[3 0 4]
[6 8 5]


Step  8 :
[1 2 7]
[3 4 0]
[6 8 5]


Step  9 :
[1 2 0]
[3 4 7]
[6 8 5]


Step  10 :
[1 0 2]
[3 4 7]
[6 8 5]


Step  11 :
[1 4 2]
[3 0 7]
[6 8 5]


Step  12 :
[1 4 2]
[3 7 0]
[6 8 5]


Step  13 :
[1 4 2]
[3 7 5]
[6 8 0]


Step  14 :
[1 4 2]
[3 7 5]
[6 0 8]


Step  15 :
[1 4 2]
[3 0 5]
[6 7 8]


Step  16 :
[1 0 2]
[3 4 5]
[6 7 8]


Step  17 :
[0 1 2]
[3 4 5]
[6 7 8]


Steps to solution:  17
Maximum frontier size:  51


(17, 51, 0)

In [4]:
results = []
arrays = [[[5,7,6],[2,4,3],[8,1,0]],
          [[7,0,8],[4,6,1],[5,3,2]],
          [[2,3,7],[1,8,0],[6,5,4]]]

for array in arrays:
    results.append([solvePuzzle(3,array,heuristics[0],prnt=False,_counter=True),
                    solvePuzzle(3,array,heuristics[1],prnt=False,_counter=True)])




In [5]:
print("         Misplaced Tiles                            Manhattan Distance")
print("Steps   Max Frontier Size   Expansions      Steps   Max Frontier Size   Expansions",)
for t in results:
    if t[0][1]<10000:
        print("",t[0][0],"         ",t[0][1],"             ",t[0][3],"         ",
                 t[1][0],"         ",t[1][1],"           ",t[1][3])
    else:
        print("",t[0][0],"        ",t[0][1],"           ",t[0][3],"        ",
                 t[1][0],"        ",t[1][1],"           ",t[1][3])


         Misplaced Tiles                            Manhattan Distance
Steps   Max Frontier Size   Expansions      Steps   Max Frontier Size   Expansions
 28          21684             63437          28          916             1529
 25          12065             25342          25          520             900
 17           431               674           17           51             83


In the table above we can see the results of running our code using both the "Misplaced Tiles" and the "Manhattan Distance" heuristics. Every row is a different initial state from which we start our search towards the goal. Both heuristics show you get the same amount of steps. This is most likely because they both have reached optimal solutions and it can't be reached in less time than that.
<br>
<br>
However, we can see there's a big difference in their "Maximum Frontier Size" and "Expansions" columns. The "Manhattan Distance" heuristic seems to be much better as it doesn't have to expand as many nodes as the "Misplaced Tiles" heuristic. It is probably due to this that the maximum frontier size is also larger when we use the "Misplaced Tiles" heuristic.
<br>
<br>
Using the "Manhattan Distance" heuristic we probably expand much fewer nodes as it is a better heuristic for evaluating how close we are to reaching a solution. While the "Misplaced Tiles" heuristic can only return 9 values, there are many more states with the highest values as there are many ways to place all the tiles in the wrong position. The "Manhattan Distance" heuristic is more subtle in telling how far from the solution you are as it takes into account not only how many tiles are misplaced but how far they are from where they should be. Therefore, with this heuristic it is easier to have the next state be "closer" than it is with just counting misplaced tiles.
<br>
<br>
Code can be accesed here:
<br>
https://gist.github.com/JoeSnow7/ccfe21edf24a4ec345e7b965e3fdcc2c